# StartUp

In [1]:
using JuMP
using Gurobi

# Constants

In [2]:
Stadiums = 1:16;
Countries = 1:48;
Days = 1:16;
Times = 1:3;

#Stadiums = 17;
#Countries = 72;
#Days = 17;
#Times = 3;



#new coefficient for round robbins that I came up with after a comment that each team plays 3 games and I thought they were 6
# RR stands for Round Robin
RR = 3;

DR1 = [[0 120 800 1080];
    [140 0 680 960];
    [800 680 0 380]; 
    [1050 960 380 0]];

DR2 = [[0 570 340 930 940 1160 1330]; 
    [570 0 420 610 510 780 1020];
    [340 420 0 940 880 1050 1220];
    [930 610 940 0 240 460 780];
    [940 510 880 240 0 650 800];
    [1160 780 1050 460 650 0 800];
    [1330 1020 1220 780 800 800 0]];

DR3 = [[0 550 490 510 1480];
    [550 0 220 310 1250]; 
    [490 220 0 100 1280];
    [510 310 100 0 1210];
    [1480 1250 1280 1210 0]];

V0 = [
    [0 140 800 1050 0 0 0 0 0 0 0 0 0 0 0 0],
    [140 0 680 960 0 0 0 0 0 0 0 0 0 0 0 0],
    [800 680 0 380 0 0 0 0 0 0 0 0 0 0 0 0], 
    [1050 960 380 0 0 0 0 0 0 0 0 0 0 0 0 0],
    [0 0 0 0 0 570 340 930 940 1160 1330 0 0 0 0 0], 
    [0 0 0 0 570 0 420 610 510 780 1020 0 0 0 0 0], 
    [0 0 0 0 340 420 0 940 880 1050 1220 0 0 0 0 0], 
    [0 0 0 0 930 610 940 0 240 460 780 0 0 0 0 0], 
    [0 0 0 0 940 510 880 240 0 650 800 0 0 0 0 0], 
    [0 0 0 0 1160 780 1050 460 650 0 800 0 0 0 0 0], 
    [0 0 0 0 1330 1020 1220 780 800 800 0 0 0 0 0 0], 
    [0 0 0 0 0 0 0 0 0 0 0 0 0 550 490 510 1480],
    [0 0 0 0 0 0 0 0 0 0 0 0 550 0 220 310 1250],
    [0 0 0 0 0 0 0 0 0 0 0 0 490 220 0 100 1280],
    [0 0 0 0 0 0 0 0 0 0 0 0 510 310 100 0 1210],
    [0 0 0 0 0 0 0 0 0 0 0 0 1480 1250 1280 1210 0]
    ]

M = 9999;
VM = [
    [M 140 800 1050 M M M M M M M M M M M M];
    [140 M 680 960 M M M M M M M M M M M M];
    [800 680 M 380 M M M M M M M M M M M M]; 
    [1050 960 380 M M M M M M M M M M M M M];
    [M M M M M 570 340 930 940 1160 1330 M M M M M]; 
    [M M M M 570 M 420 610 510 780 1020 M M M M M];
    [M M M M 340 420 M 940 880 1050 1220 M M M M M]; 
    [M M M M 930 610 940 M 240 460 780 M M M M M]; 
    [M M M M 940 510 880 240 M 650 800 M M M M M]; 
    [M M M M 1160 780 1050 460 650 M 800 M M M M M]; 
    [M M M M 1330 1020 1220 780 800 800 M M M M M M]; 
    [M M M M M M M M M M M M 550 490 510 1480];
    [M M M M M M M M M M M 550 M 220 310 1250];
    [M M M M M M M M M M M 490 220 M 100 1280];
    [M M M M M M M M M M M 510 310 100 M 1210];
    [M M M M M M M M M M M 1480 1250 1280 1210 M]
    ]

V = [[0 140 800 1050 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 ];
    [140 0 680 960 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 ];
    [800 680 0 380 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 ];
    [1050 960 380 0 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 ];
    [1000000 1000000 1000000 1000000 0 570 340 930 940 1160 1330 1000000 1000000 1000000 1000000 1000000 ]; 
    [1000000 1000000 1000000 1000000 570 0 420 610 510 780 1020 1000000 1000000 1000000 1000000 1000000 ]; 
    [1000000 1000000 1000000 1000000 340 420 0 940 880 1050 1220 1000000 1000000 1000000 1000000 1000000 ]; 
    [1000000 1000000 1000000 1000000 930 610 940 0 240 460 780 1000000 1000000 1000000 1000000 1000000 ]; 
    [1000000 1000000 1000000 1000000 940 510 880 240 0 650 800 1000000 1000000 1000000 1000000 1000000 ]; 
    [1000000 1000000 1000000 1000000 1160 780 1050 460 650 0 800 1000000 1000000 1000000 1000000 1000000 ]; 
    [1000000 1000000 1000000 1000000 1330 1020 1220 780 800 800 0 1000000 1000000 1000000 1000000 1000000 ]; 
    [1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 0 550 490 510 1480];
    [1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 550 0 220 310 1250];
    [1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 490 220 0 100 1280];
    [1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 510 310 100 0 1210];
    [1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1000000 1480 1250 1280 1210 0]]


LD = [1 3 4 4 4 4 4 4 4 4 4 4 4 6 6 6 6];

# Gurobi Model Creation

In [3]:
model = Model(Gurobi.Optimizer);


Set parameter Username
Set parameter LicenseID to value 2685810
Academic license - for non-commercial use only - expires 2026-07-09


## Variables

In [4]:
@variable(model, X[c in Countries, s in Stadiums], Bin);
@variable(model, Y[c1 in Countries, c2 in Countries, d in Days], Bin);  
@variable(model, Z[c in Countries, d in Days, t in Times], Bin);

## Objective

In [5]:
@objective(model, Min, sum(
    V[s1, s2] * X[c, s1] * X[c, s2]
    for c in Countries, s1 in 1:16, s2 in 1:16))


280 X[1,1]*X[1,2] + 1600 X[1,1]*X[1,3] + 2100 X[1,1]*X[1,4] + 2000000 X[1,1]*X[1,5] + 2000000 X[1,1]*X[1,6] + 2000000 X[1,1]*X[1,7] + 2000000 X[1,1]*X[1,8] + 2000000 X[1,1]*X[1,9] + 2000000 X[1,1]*X[1,10] + 2000000 X[1,1]*X[1,11] + 2000000 X[1,1]*X[1,12] + 2000000 X[1,1]*X[1,13] + 2000000 X[1,1]*X[1,14] + 2000000 X[1,1]*X[1,15] + 2000000 X[1,1]*X[1,16] + 1360 X[1,2]*X[1,3] + 1920 X[1,2]*X[1,4] + 2000000 X[1,2]*X[1,5] + 2000000 X[1,2]*X[1,6] + 2000000 X[1,2]*X[1,7] + 2000000 X[1,2]*X[1,8] + 2000000 X[1,2]*X[1,9] + 2000000 X[1,2]*X[1,10] + 2000000 X[1,2]*X[1,11] + 2000000 X[1,2]*X[1,12] + 2000000 X[1,2]*X[1,13] + 2000000 X[1,2]*X[1,14] + 2000000 X[1,2]*X[1,15] + 2000000 X[1,2]*X[1,16] + 760 X[1,3]*X[1,4] + [[...5700 terms omitted...]] + 2000000 X[48,8]*X[48,15] + 2000000 X[48,8]*X[48,16] + 1300 X[48,9]*X[48,10] + 1600 X[48,9]*X[48,11] + 2000000 X[48,9]*X[48,12] + 2000000 X[48,9]*X[48,13] + 2000000 X[48,9]*X[48,14] + 2000000 X[48,9]*X[48,15] + 2000000 X[48,9]*X[48,16] + 1600 X[48,10]*X[48

# Constraints

### Model Breakers

In [6]:
# Round Robbin Matches
@constraint(model, [c in Countries], 
    sum(Z[c, d, t] for d in Days, t in Times) == RR);

# Match ensuring Constraints
@constraint(model, [c1 in Countries, c2 in Countries, d in Days],
        sum(X[c1, s] * X[c2, s] for s in Stadiums) <= Y[c1, c2, d]);

@constraint(model, [c1 in Countries, c2 in Countries, d in Days],
        sum(Z[c1, d, t] * Z[c2, d, t] for t in Times) <= Y[c1, c2, d]);



### Special Constraints Outlined by Rubric

In [7]:
# MX opening match on June 11 8PM at Estadio Azteca 
@constraint(model, Z[1, 1, 3] == 1);
@constraint(model, X[1, 5] == 1);


# USA opening match on June 12 8PM at SoFi Stadium 
@constraint(model, Z[2, 2, 3] == 1);
@constraint(model, X[1, 5] == 1);

# CAN opening match on June 12 8PM at BMO Field 
@constraint(model, Z[3, 2, 3] == 1);
@constraint(model, X[3, 12] == 1);

# Daily Match Limit
@constraint(model, [d in 3:maximum(Days)], sum(Y[i,j,d] for i in Countries, j in Countries) <= LD[d]);

# Each Stadium Hosts at least one match
@constraint(model, [s in Stadiums], sum(X[c, s] for c in Countries) >= 1);

# Match distribution accross stadiums is approximately equal

@constraint(model, [s in 1:length(Stadiums)-1], 
        sum(X[c, s] for c in Countries) - sum(X[c, s+1] for c in Countries) <= 1);

@constraint(model, [s in 1:length(Stadiums)-1], 
        sum(X[c, s+1] for c in Countries) - sum(X[c, s] for c in Countries) <= 1);

# Teams don't play more than once per day
@constraint(model, [c in Countries, d in Days], sum(Z[c, d, t] for t in Times) <= 1);

# Two days of rest between matches
@constraint(model, [c in Countries, d in 1:(maximum(Days)-2)], 
        sum(Z[c, d, t] for t in Times) +
        sum(Z[c, d+1, t] for t in Times) +
        sum(Z[c, d+2, t] for t in Times) <= 1);

# No overlapping matches of same stadium same time
@constraint(model, [s in Stadiums, d in Days, t in Times], sum(X[c, s] * Z[c, d, t] for c in Countries) <= 2);

# Host nations only play at 8 PM
#MX
@constraint(model, sum(Z[1, d, 1] for d in Days) + sum(Z[1, d, 2] for d in Days) == 0);

#USA
@constraint(model, sum(Z[2, d, 1] for d in Days) + sum(Z[2, d, 2] for d in Days) == 0);

#CAN
@constraint(model, sum(Z[3, d, 1] for d in Days) + sum(Z[3, d, 2] for d in Days) == 0);

#Symmetry
@constraint(model, [i in Countries, j in Countries], 
    sum(Y[i, j, d] + Y[j, i, d] for d in Days) <= 1); 

# Team doesn't play itself, deprecated constraint but I still wanted to keep to keep it around in my notes 
@constraint(model, [c in Countries], sum(Y[c, c, d] for d in Days) == 0);

# Matches don't repeat
@constraint(model, [i in Countries, j in Countries], sum(Y[i, j, d] for d in Days) <= 1);

### Optimization

In [8]:
optimize!(model)



Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6213 rows, 39936 columns and 157254 nonzeros
Model fingerprint: 0x39b41622
Model has 5760 quadratic objective terms
Model has 74496 quadratic constraints
Variable types: 0 continuous, 39936 integer (39936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+02, 4e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+00]
  QRHS range       [2e+00, 2e+00]
Presolve added 1536 rows and 0 columns
Presolve time: 0.08s

Explored 0 nodes (0 simplex iterations) in 0.09 seconds (0.13 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is i